In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import logging
#
import airavata_cerebrum.model.setup as cbm_setup
import airavata_cerebrum.view.tree as cbm_tree
import airavata_cerebrum.model.recipe as cbm_recipe
import wgn_sleep.model as wgn_model

In [3]:
import pathlib
model_base_dir = "./"
model_name = "wgn_sleep"
custom_mod_file = "./wgn_sleep/recipe/custom_mod.json"
recipe_dir = "./wgn_sleep/recipe/"
recipe_files={"recipe": ["recipe.json"], "templates": ["recipe_template.json"] }
recipe_main_file = pathlib.Path(recipe_dir, recipe_files["recipe"][0])

In [4]:
import IPython.display
import json
with open(recipe_main_file) as ifx:
    config_dict = json.load(ifx)
IPython.display.JSON(config_dict)

<IPython.core.display.JSON object>

In [5]:
mdr_setup = cbm_setup.init_model_setup(
    name=model_name,
    model_base_dir=model_base_dir,
    recipe_files=recipe_files,
    recipe_dir=recipe_dir,
)

Loading View Templates


In [6]:
import IPython.display
import json
with open(custom_mod_file) as ifx:
    custom_mod_dict = json.load(ifx)
IPython.display.JSON(custom_mod_dict)

<IPython.core.display.JSON object>

In [7]:
custom_mods = cbm_recipe.netstruct_from_file(custom_mod_file)
cmod_view = cbm_tree.NetworkTreeView(custom_mods, left_width="55%")
cmod_view.build()
cmod_view.layout

TwoByTwoLayout(children=(Tree(layout=Layout(grid_area='top-left', width='55%'), multiple_selection=False, node…

In [39]:
model_recipe = cbm_recipe.ModelRecipe(
    recipe_setup=mdr_setup,
    region_mapper=wgn_model.WGNRegionMapper,                                                                                                                                                                                                         
    neuron_mapper=wgn_model.WGNNeuronMapper,                                                                                                                                                                                                         
    connection_mapper=wgn_model.WGNConnectionMapper,                                                                                                                                                                                                     
    network_builder=wgn_model.WGNNetworkBuilder,                                                                                                                                                                                                   
    custom_mod="./wgn_sleep/recipe/custom_mod.json",                                                                                                                                                                                                               
    save_flag=True,
)                                                                                                                                                                                                                                   
# logging.basicConfig(level=logging.INFO)